Injection code
    
    import pickle
    with open('/tmp/data.pkl', 'wb') as f:
        data = dict(
            parameters=parameters,
            filenames=filenames,
        )
        pickle.dump(data, f)

    from os import execvp
    os.environ['BAZEL_RUNFILES'] = os.getcwd()
    os.environ['PATH'] = '/home/eacousineau/bin:/home/eacousineau/.local/bin:'
    sys.path.insert(0, os.path.dirname(__file__))
    os.environ['PYTHONPATH'] = ':'.join(sys.path)
    execvp('jupyter', ['jupyter', 'notebook'])
    exit(1)


In [ ]:
import os
os.chdir(os.environ['BAZEL_RUNFILES'])

In [ ]:
from mkdoc import *

In [ ]:
import pickle

In [ ]:
with open('/tmp/data.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
filenames = data['filenames']
parameters = data['parameters']

In [ ]:
quiet = False
if True:
    includes = list(map(drake_genfile_path_to_include_path, filenames))
    include_map = FileDict(zip(filenames, includes))
    # TODO(eric.cousineau): Sort files based on include path?
    with NamedTemporaryFile('w') as include_file:
        for include in includes:
            include_file.write("#include \"{}\"\n".format(include))
        include_file.flush()
        if not quiet:
            print("Parse header...", file=sys.stderr)
        index = cindex.Index(
            cindex.conf.lib.clang_createIndex(False, True))
        tu = index.parse(include_file.name, parameters)

    if not quiet:
        print("Extract relevant symbols...", file=sys.stderr)
    output = []
    extract(include_map, tu.cursor, '', output)

In [ ]:
symbols = sorted(output, key=Symbol.sorting_key)

In [ ]:
sym_map = {"{}:{}".format(s.include, s.line): s for s in symbols}

In [ ]:

s = sym_map['drake/multibody/rigid_body_tree.h:161']  # No definition


In [ ]:
n = s.node

In [ ]:
n.get_definition().raw_comment

In [ ]:
n.type.spelling

In [ ]:
ps = [
    [
        ('bad', sym_map['drake/multibody/ik_options.h:6']),
        ('good', sym_map['drake/multibody/rigid_body_tree.h:109']),
    ],
    [
        ('bad', sym_map['drake/multibody/kinematics_cache-inl.h:71']),
        ('good', sym_map['drake/multibody/kinematics_cache.h:152']),
    ],
]
for p in ps:
    for rar, s in p:
        print(rar)
        n = s.node
        print(n.type.spelling)
        print(n.location)
        print(len(n.raw_comment))
        print(n.canonical.location)
        print(n.is_definition())
        print("---")
    print("\n\n")